## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [1]:
cd ../dataprocessing

/usr/local/google/home/carverforbes/activity-recognition/location/lbs/activity/audioset/dataprocessing


## Call the import statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [2]:
# import statements
import audio_processing as ap

import functools
import os
import sys
from absl import logging

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

print("Ran the import statements.")

Ran the import statements.


In [3]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [4]:
# arguments for audio_processing
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
filename = 'gunshot_50_50'
notebook = 'training_rms_cnn'
labels = ['Gunshot, gunfire']
available_features = ['chroma_stft',
                       'chroma_cqt',
                       'chroma_cens',
                       'melspectrogram',
                       'mfcc',
                       'rms',
                       'spectral_centroid',
                       'spectral_bandwidth',
                       'spectral_contrast',
                       'spectral_flatness',
                       'spectral_rolloff',
                       'poly_features',
                       'tonnetz',
                       'zero_crossing_rate'
]
features_to_extract = ['rms']

In [5]:
# Import the dataset
df = ap.output_df(src_dir, dest_dir, filename, labels, features_to_extract, False)

INFO:absl:The set has 23728 examples
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:a

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Already Downloaded
INFO:absl:Alre

...

INFO:absl:(20511, 13255.407695)
INFO:absl:extracted features
INFO:absl:(20512, 13255.967675)
INFO:absl:extracted features
INFO:absl:(20513, 13256.576036)
INFO:absl:extracted features
INFO:absl:(20514, 13257.142155)
INFO:absl:extracted features
INFO:absl:(20515, 13257.748313)
INFO:absl:extracted features
INFO:absl:(20516, 13258.327305)
INFO:absl:extracted features
INFO:absl:(20517, 13258.936232)
INFO:absl:extracted features
INFO:absl:(20518, 13259.530014)
INFO:absl:extracted features
INFO:absl:(20519, 13260.153579)
INFO:absl:extracted features
INFO:absl:(20520, 13260.746582)
INFO:absl:extracted features
INFO:absl:(20521, 13261.336694)
INFO:absl:extracted features
INFO:absl:(20522, 13261.892668)
INFO:absl:extracted features
INFO:absl:(20523, 13262.473017)
INFO:absl:extracted features
INFO:absl:(20524, 13263.130459)
INFO:absl:extracted features
INFO:absl:(20525, 13263.789521)
INFO:absl:extracted features
INFO:absl:(20526, 13264.36077)
INFO:absl:extracted features
INFO:absl:(20527, 13264.9

INFO:absl:(20646, 13339.944632)
INFO:absl:extracted features
INFO:absl:(20647, 13340.50969)
INFO:absl:extracted features
INFO:absl:(20648, 13341.121954)
INFO:absl:extracted features
INFO:absl:(20649, 13341.7673)
INFO:absl:extracted features
INFO:absl:(20650, 13342.437244)
INFO:absl:extracted features
INFO:absl:(20651, 13343.074944)
INFO:absl:extracted features
INFO:absl:(20652, 13343.652927)
INFO:absl:extracted features
INFO:absl:(20653, 13344.353862)
INFO:absl:extracted features
INFO:absl:(20654, 13344.954417)
INFO:absl:extracted features
INFO:absl:(20655, 13345.55277)
INFO:absl:extracted features
INFO:absl:(20656, 13346.102965)
INFO:absl:extracted features
INFO:absl:(20657, 13346.792549)
INFO:absl:extracted features
INFO:absl:(20658, 13347.40811)
INFO:absl:extracted features
INFO:absl:(20659, 13348.038481)
INFO:absl:extracted features
INFO:absl:(20660, 13348.599161)
INFO:absl:extracted features
INFO:absl:(20661, 13349.299275)
INFO:absl:extracted features
INFO:absl:(20662, 13349.42512

INFO:absl:(20781, 13423.914554)
INFO:absl:extracted features
INFO:absl:(20782, 13424.534419)
INFO:absl:extracted features
INFO:absl:(20783, 13425.172784)
INFO:absl:extracted features
INFO:absl:(20784, 13425.746525)
INFO:absl:extracted features
INFO:absl:(20785, 13426.389534)
INFO:absl:extracted features
INFO:absl:(20786, 13426.706202)
INFO:absl:extracted features
INFO:absl:(20787, 13427.334757)
INFO:absl:extracted features
INFO:absl:(20788, 13427.965808)
INFO:absl:extracted features
INFO:absl:(20789, 13428.571981)
INFO:absl:extracted features
INFO:absl:(20790, 13429.120199)
INFO:absl:extracted features
INFO:absl:(20791, 13429.740505)
INFO:absl:extracted features
INFO:absl:(20792, 13430.333484)
INFO:absl:extracted features
INFO:absl:(20793, 13430.92354)
INFO:absl:extracted features
INFO:absl:(20794, 13431.467262)
INFO:absl:extracted features
INFO:absl:(20795, 13432.074762)
INFO:absl:extracted features
INFO:absl:(20796, 13432.839826)
INFO:absl:extracted features
INFO:absl:(20797, 13433.4

INFO:absl:extracted features
INFO:absl:(20916, 13507.999898)
INFO:absl:extracted features
INFO:absl:(20917, 13508.592614)
INFO:absl:extracted features
INFO:absl:(20918, 13509.245747)
INFO:absl:extracted features
INFO:absl:(20919, 13509.906926)
INFO:absl:extracted features
INFO:absl:(20920, 13510.486403)
INFO:absl:extracted features
INFO:absl:(20921, 13511.062511)
INFO:absl:extracted features
INFO:absl:(20922, 13511.702803)
INFO:absl:extracted features
INFO:absl:(20923, 13512.26748)
INFO:absl:extracted features
INFO:absl:(20924, 13512.878249)
INFO:absl:extracted features
INFO:absl:(20925, 13513.474014)
INFO:absl:extracted features
INFO:absl:(20926, 13514.056927)
INFO:absl:extracted features
INFO:absl:(20927, 13514.836756)
INFO:absl:extracted features
INFO:absl:(20928, 13515.615682)
INFO:absl:extracted features
INFO:absl:(20929, 13516.272445)
INFO:absl:extracted features
INFO:absl:(20930, 13516.814102)
INFO:absl:extracted features
INFO:absl:(20931, 13517.430932)
INFO:absl:extracted featu

INFO:absl:extracted features
INFO:absl:(21051, 13592.900721)
INFO:absl:extracted features
INFO:absl:(21052, 13593.555331)
INFO:absl:extracted features
INFO:absl:(21053, 13594.194028)
INFO:absl:extracted features
INFO:absl:(21054, 13594.861563)
INFO:absl:extracted features
INFO:absl:(21055, 13595.463509)
INFO:absl:extracted features
INFO:absl:(21056, 13596.213582)
INFO:absl:extracted features
INFO:absl:(21057, 13596.792103)
INFO:absl:extracted features
INFO:absl:(21058, 13597.436159)
INFO:absl:extracted features
INFO:absl:(21059, 13598.09102)
INFO:absl:extracted features
INFO:absl:(21060, 13598.665302)
INFO:absl:extracted features
INFO:absl:(21061, 13599.312577)
INFO:absl:extracted features
INFO:absl:(21062, 13599.925358)
INFO:absl:extracted features
INFO:absl:(21063, 13600.508737)
INFO:absl:extracted features
INFO:absl:(21064, 13601.135814)
INFO:absl:extracted features
INFO:absl:(21065, 13601.756901)
INFO:absl:extracted features
INFO:absl:(21066, 13602.430471)
INFO:absl:extracted featu

In [6]:
df.head()

,label,rms
0,1,"[[0.024051616, 0.02332899, 0.023757856, 0.0242..."
1,1,"[[0.018282544, 0.018028082, 0.017703537, 0.016..."
2,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1,"[[0.04337968, 0.04965238, 0.052755404, 0.05585..."
4,0,"[[0.0066741644, 0.00669201, 0.0066719446, 0.00..."


In [8]:
train_df_rms = train_df_rms.reindex(
    np.random.permutation(train_df_rms.index))
train_df_rms.head()

,label,rms
17974,0,"[[0.20318848, 0.19580616, 0.19221099, 0.184523..."
18145,0,"[[0.07592033, 0.074876815, 0.06766564, 0.05775..."
9443,0,"[[0.0056607127, 0.0056455387, 0.0055667395, 0...."
9392,0,"[[0.0032344405, 0.003348331, 0.0033364997, 0.0..."
14025,0,"[[0.09453177, 0.09391603, 0.09427221, 0.093018..."


In [9]:
# temp bug fix for rows with None features
# also deletes the outer array in the features
rows_with_none = []
print(train_df_rms.size / 2)
# print(train_df_rms.rms[0])
for i in train_df_rms.index:
    if train_df_rms.rms[i] is None:
        rows_with_none.append(i)
        continue
    train_df_rms.rms[i] = train_df_rms.rms[i][0]
print(rows_with_none)
train_df_rms = train_df_rms.drop(rows_with_none)
train_df_rms.size / 2
train_df_rms.head(10000)

21112.0


<ipython-input-9-f2f371f1575e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_rms.rms[i] = train_df_rms.rms[i][0]


[2383, 121, 15656]


,label,rms
17974,0,"[0.20318848, 0.19580616, 0.19221099, 0.1845238..."
18145,0,"[0.07592033, 0.074876815, 0.06766564, 0.057757..."
9443,0,"[0.0056607127, 0.0056455387, 0.0055667395, 0.0..."
9392,0,"[0.0032344405, 0.003348331, 0.0033364997, 0.00..."
14025,0,"[0.09453177, 0.09391603, 0.09427221, 0.0930183..."
...,...,...
9980,0,"[0.113159254, 0.11508956, 0.11434702, 0.113026..."
215,1,"[0.045642473, 0.047532577, 0.049895942, 0.0504..."
10260,0,"[0.054532364, 0.05730033, 0.064113945, 0.07247..."
13597,0,"[0.1849465, 0.18557855, 0.18045238, 0.18438655..."


In [10]:
X = np.array(train_df_rms.rms.tolist(), dtype=object)
y = np.array(train_df_rms.label.tolist())

In [11]:
# Temp bug fix for having different sized features
temp_x = []
temp_y = []
count = 0
for arr, label in zip(X, y):
    if arr is None or len(arr) < 431:
#         print('hey look at me', len(arr))
        count += 1
        continue
    temp_x.append(arr)
    temp_y.append(label)
X = np.array(temp_x, dtype=object)
y = np.array(temp_y)
print(count)

1651


In [ ]:
def make_dir(path):
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
            return False
    return True

In [12]:
# Define the plotting function.
def plot_curve(epochs, hist, dest_path, notebook_filename, dataset_filename, list_of_metrics, list_of_hyperparameters):
    """Plot a curve of one or more classification metrics vs. epoch and save it to path."""  
    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    
    make_dir(dest_path)    
    path = os.path.join(dest_path, notebook_filename)
    make_dir(path)
    path = os.path.join(path, dataset_filename)
    make_dir(path)
        
    list_of_hyperparameters_temp = [str(item) for item in list_of_hyperparameters]
    filename = '_'.join(list_of_hyperparameters_temp)
    path = os.path.join(path, filename + '.png')
    plt.savefig(path, bbox_inches='tight')
    
    return plt


print("Defined the plot_curve function.")

Defined the plot_curve function.


In [13]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)

In [14]:
# trying to fix bug:
# ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
# It worked!!!
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_val = K.cast_to_floatx(x_val)
y_val = K.cast_to_floatx(y_val)

ModuleNotFoundError: No module named 'keras'

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
y_train = y_train.reshape(y_train.shape[0], 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], 1)

In [ ]:
# print(x_train)
print(x_train.shape)

## Training 1

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()
              
    model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=43, padding='valid',
                                  activation='relu', input_shape=(431, 1),
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Conv1D_1'))
    
    model.add(tf.keras.layers.Flatten(name='Flatten_1'))
    
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=20, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Hidden1'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 500
batch_size = 25
classification_threshold = 0.70
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           label_name]

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 2
Convoluted Neural Network:
* Conv1D Layer 1:
* MaxPooling1D 1:
* Conv1D Layer 1:
* MaxPooling1D 1:
* Flatten Layer 1: 
* Hidden Layer 1: 
* Hidden Layer 2:
* Ouput Layer: 1 node
Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001
* Epochs: 
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()
              
    model.add(tf.keras.layers.Conv1D(filters=5, kernel_size=43, padding='valid',
                                  activation='relu', input_shape=(431, 1),
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Conv1D_1'))
    
    model.add(tf.keras.layers.MaxPooling1D(pool_size=20,
                                           strides=19,
                                           padding='valid',
                                           name='MaxPooling1D_1'))
    
    model.add(tf.keras.layers.Flatten(name='Flatten_1'))
    
    # Implement L1 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=100, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Hidden1'))
    
#     # Implement L1 regularization in the second hidden layer.
#     model.add(tf.keras.layers.Dense(units=20, 
#                                   activation='relu',
#                                   kernel_regularizer=tf.keras.regularizers.l1(0.001),
#                                   name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
label_name = "label"


# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, 'example_dest_dir/training_rms_cnn/', filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], 'example_dest_dir/training_rms_cnn/', filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 3
Convoluted Neural Network:
* Conv1D Layer 1:
* MaxPooling1D 1:
* Conv1D Layer 1:
* MaxPooling1D 1:
* Flatten Layer 1: 
* Hidden Layer 1: 
* Hidden Layer 2:
* Ouput Layer: 1 node
Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001
* Epochs: 
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()
              
    model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=43, padding='valid',
                                  activation='relu', input_shape=(431, 1),
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Conv1D_1'))
    
    model.add(tf.keras.layers.MaxPooling1D(pool_size=20,
                                           strides=19,
                                           padding='valid',
                                           name='MaxPooling1D_1'))
    
    model.add(tf.keras.layers.Conv1D(filters=1, kernel_size=2, padding='valid',
                                  activation='relu', input_shape=(431, 1),
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Conv1D_2'))
    
    model.add(tf.keras.layers.Flatten(name='Flatten_1'))
    
    # Implement L1 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=100, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                                  name='Hidden1'))
    
#     # Implement L1 regularization in the second hidden layer.
#     model.add(tf.keras.layers.Dense(units=20, 
#                                   activation='relu',
#                                   kernel_regularizer=tf.keras.regularizers.l1(0.001),
#                                   name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.65
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           label_name]

# Here is the updated definition of METRICS:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=classification_threshold),
    tf.keras.metrics.Precision(thresholds=classification_threshold, name='precision'),
    tf.keras.metrics.Recall(thresholds=classification_threshold, name="recall"),
]

# Create model
my_model = create_model(learning_rate, METRICS)

# View the model's structure.
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, dest_dir, notebook, filename, list_of_metrics_to_plot, list_of_hyperparameters)
plot_curve(epochs, hist, dest_dir, notebook, filename, ['loss', 'val_loss'], list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])